In [ ]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS

In [ ]:
attcs = ATTCS()

In [ ]:
await attcs.start_task

In [ ]:
logger = salobj.Controller("Script", index=1)

In [ ]:
await logger.start_task

In [ ]:
r = salobj.Remote(attcs.domain, "GenericCamera", 1)

In [9]:
await attcs.athexapod.cmd_moveToPosition.set_start(x=-4.30, y=1.20, z=0.50+0.034, u=0.35, v=0.22, w=0., timeout=30.)

In [ ]:
ref_pos = [-4.30, 1.20, 0.50, 0.35, 0.22, 0.]
offset = 1.5

In [ ]:
await r.cmd_stopLiveView.start()

In [ ]:
logger.evt_logMessage.set_put(message="Test Starting")
# intra
print("Moving hexapod intra")
await attcs.athexapod.cmd_moveToPosition.set_start(x=ref_pos[0], 
                                                   y=ref_pos[1], 
                                                   z=ref_pos[2]-offset, 
                                                   u=ref_pos[3], 
                                                   v=ref_pos[4], 
                                                   w=ref_pos[5], 
                                                   timeout=30.)
print("Taking image")
await r.cmd_takeImages.set_start(numImages=1,
                                 expTime=15.,
                                 shutter=True,
                                 imageSequenceName='HIP25531_intra')
# extra
print("Moving hexapod extra")
await attcs.athexapod.cmd_moveToPosition.set_start(x=ref_pos[0], 
                                                   y=ref_pos[1], 
                                                   z=ref_pos[2]+offset, 
                                                   u=ref_pos[3], 
                                                   v=ref_pos[4], 
                                                   w=ref_pos[5], 
                                                   timeout=30.)
print("Taking image")
await r.cmd_takeImages.set_start(numImages=1,
                                 expTime=15.,
                                 shutter=True,
                                 imageSequenceName='HIP25531_extra')
print("Done")
logger.evt_logMessage.set_put(message="Test Ending")

In [ ]:
await r.cmd_startLiveView.set_start(expTime=0.5)

In [ ]:
await salobj.set_summary_state(attcs.athexapod, salobj.State.STANDBY)

In [8]:
await asyncio.sleep(5)
await salobj.set_summary_state(attcs.athexapod, salobj.State.ENABLED, settingsToApply="current")

[<State.DISABLED: 1>, <State.ENABLED: 2>]

In [7]:
await attcs.athexapod.cmd_start.set_start(settingsToApply="current")